**Imports**

In [1]:
import pandas as pd
import numpy as np
import glob
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.image as mpimg
import os
import numpy
from sklearn.preprocessing import LabelEncoder

**Reading files**

In [2]:
X_fileName = []
y_val = []

In [3]:
for fileName in sorted(glob.iglob('near-infrared/**/train_pose/**/**', recursive=False)):
    X_fileName.append(fileName)
    y_val.append(fileName.split("\\")[-2])

In [4]:
myDf = pd.DataFrame()
myDf['fileName'] = X_fileName
myDf['label'] = y_val

In [5]:
myLabelEncoder = LabelEncoder()
myDf['label_val'] = myLabelEncoder.fit_transform(myDf['label'])

**PreProcessing**

In [6]:
def pointremove(image):
    
    img=image.copy()
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=4)
    sizes = stats[:, -1]
    for i in range(1, nb_components):
        if sizes[i]<100 :
            img[output==i]=0
    return img

In [7]:
def maxmeanpreprop(path):
    img1 = cv2.imread(path)
    g_img = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img = g_img.copy()
    ret1,th1 = cv2.threshold(g_img,40,255,cv2.THRESH_BINARY)
    th1=pointremove(th1)
    blur = cv2.GaussianBlur(th1,(5,5),0)
    ret2,th2 = cv2.threshold(blur,130,255,cv2.THRESH_OTSU)

    a_img=th2&img
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(a_img, connectivity=4)
    sizes = stats[:, -1]

    max_label = 0
    max_mean = 0

    for i in range(1, nb_components):
        mean=np.mean(a_img[output==i])
        if mean > max_mean and sizes[i]>1000 :
            max_label = i
            max_mean = mean

    img3 = np.zeros(output.shape)
    img3[output == max_label] = 255
    new_image = img3[stats[max_label][1]:stats[max_label][1]+stats[max_label][3],stats[max_label][0]:stats[max_label][0]+stats[max_label][2]]
    image_for_input = cv2.resize(new_image, (100, 100), interpolation = cv2.INTER_NEAREST)
    return image_for_input

**Creating my generator**

In [8]:
def my_generator(features, batch_size=128):
    # Create empty arrays to contain batch of features and labels#
    batch_features = np.zeros((batch_size, 100, 100, 1))
    batch_labels = np.zeros((batch_size))
    while True:
        for i in range(batch_size):
            # choose random index in features
            index = np.random.choice(len(features),1)
            batch_features[i] = maxmeanpreprop(features.loc[index[0]]['fileName']).reshape([100, 100, 1])
            batch_labels[i] = features.loc[index[0]]['label_val']
        yield batch_features, batch_labels

**Model** 

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras import backend as K

Using TensorFlow backend.
C:\Users\akkir\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\akkir\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\akkir\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\akkir\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [10]:

def MyModel(input_shape):
    
    model = Sequential()

    model.add(Lambda(lambda x: x / 255 - 0.5, input_shape=input_shape))

    model.add(Conv2D(24, 5, 5, activation='relu', subsample=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(36, 5, 5, activation='relu', subsample=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(48, 5, 5, activation='relu', subsample=(2, 2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(64, 3, 3, activation='relu'))
    model.add(Dropout(0.1))

    model.add(Conv2D(64, 3, 3, activation='relu'))
    model.add(Dropout(0.1))

    model.add(Flatten())

    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(10, activation='relu'))
    model.add(Dense(16, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  

    return model

In [11]:
myMod = MyModel([100, 100, 1])

Instructions for updating:
Colocations handled automatically by placer.


C:\Users\akkir\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="relu", strides=(2, 2))`
  import sys
C:\Users\akkir\anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="relu", strides=(2, 2))`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\akkir\anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="relu", strides=(2, 2))`
  del sys.path[0]
C:\Users\akkir\anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  app.launch_new_instance()
C:\Users\akkir\anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`


In [12]:
myMod.fit_generator(my_generator(myDf), samples_per_epoch=100, nb_epoch=40)

C:\Users\akkir\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """Entry point for launching an IPython kernel.
C:\Users\akkir\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=100, epochs=40)`
  """Entry point for launching an IPython kernel.


Epoch 1/40
100/100 [==============================] - 152s 2s/step - loss: 2.6363 - accuracy: 0.1620
Epoch 2/40
100/100 [==============================] - 151s 2s/step - loss: 2.1795 - accuracy: 0.3119
Epoch 3/40
100/100 [==============================] - 152s 2s/step - loss: 1.7147 - accuracy: 0.4389
Epoch 4/40
100/100 [==============================] - 148s 1s/step - loss: 1.4030 - accuracy: 0.5332
Epoch 5/40
100/100 [==============================] - 148s 1s/step - loss: 1.1831 - accuracy: 0.6061
Epoch 6/40
100/100 [==============================] - 147s 1s/step - loss: 1.0304 - accuracy: 0.6700
Epoch 7/40
100/100 [==============================] - 147s 1s/step - loss: 0.9132 - accuracy: 0.7028
Epoch 8/40
100/100 [==============================] - 147s 1s/step - loss: 0.8131 - accuracy: 0.7434
Epoch 9/40
100/100 [==============================] - 155s 2s/step - loss: 0.7277 - accuracy: 0.7741
Epoch 10/40
100/100 [==============================] - 154s 2s/step - loss: 0.6548 - accura

In [187]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
def testing(input_size):
    X=np.zeros((input_size,100,100,1))
    Y=np.zeros((input_size))
    X_file=[]
    Y_va=[]
    for fileName in sorted(glob.iglob('near-infrared/**/test_pose/**/**', recursive=False)):
        X_file.append(fileName)
        Y_va.append(fileName.split("\\")[-2])
    Y1=myLabelEncoder.transform(Y_va)    
    k=-1
    for i in range(len(X_file)):
        k=k+1
        if (k==input_size):
            break
        image=maxmeanpreprop(X_file[i]).reshape(100,100,1)
        X[i]=image
        Y[i]=Y1[i]
        
    prediction = myMod.predict(X)
    print(prediction[3])
    compare_x=[]
    
    for i in range(input_size):
        result=np.where(prediction[i] == max(prediction[i]))
        if result[0] !=None:
            compare_x.append(result[0][0])
        else :
            compare_x.append(-1)        
    accuracy = accuracy_score(Y,compare_x)
    print('Accuracy: %f' % accuracy)
    f1 = f1_score(Y,compare_x,pos_label='positive',average='micro')
    print('F1 score: %f' % f1)
    print("Confusion Matrix :")
    print(confusion_matrix(compare_x,Y))
    print(compare_x)
    print(Y)
    return

In [189]:
testing(1000)

[0.10944539 0.05764138 0.06685384 0.05657857 0.00677484 0.04318579
 0.1726622  0.17781255 0.02331885 0.13217513 0.02231963 0.02620497
 0.0059756  0.06674547 0.03114922 0.00115662]
Accuracy: 0.922000
F1 score: 0.922000
Confusion Matrix :
[[67  6  0  0  0  0  0  0  0  8  0  0  0  0  0  0]
 [ 1 69  0  0  0  0  0  0  0  0  0  0  0  0  3  0]
 [ 3  0 48  1  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 1  0  0 74  0  0  1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 77  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  2 74  0  0  0  0  0  0  0  4  0  0]
 [ 1  0  0  0  0  0 76  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0 75  1  0  0  0  0  1  0  0]
 [ 0  5  0  0  0  0  0  2 56  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0 32  0  0  0  0  0  0]
 [ 1  0  2  0  0  0  2  0  0  0 55  0  0  0  0  0]
 [ 1  0  0  0  0  1  0  0  0  0  1 57  0  0  1  0]
 [ 3  0  0  0  1  0  0  1  1  0  0  1 56  0  1  0]
 [ 0  0  0  0  0  4  0  0  0  0  0  0  0 35  1  0]
 [ 0  0  0  0  0  0  1  2  4  0  0  0  0  0 34  0